# EJERCICIOS

In [55]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, current_date, year, monotonically_increasing_id,  avg, min, coalesce
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType

spark = SparkSession.builder.master("local[*]").appName("pyspark_rdd").getOrCreate()
#spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

## EJERCICIO 0
En un documento word haz una lista de las diferentes operaciones con una breve descripción de lo que hace y un ejemplo de como se utiliza

## EJERCICIO 1
Realiza las siguientes operaciones:
* Importa el csv de "data/WorldCupPlayers.csv" (que deduzca el esquema)
* Visualiza los datos
* ¿Que tipo de datos contiene cada variable?
* ¿Cuantos registros hay?
* Obtén los principales estadísticos de Position
* Selecciona y muestra los "Team initials" diferentes que hay ¿Cuántos hay?
* ¿Cuantos partidos con el ID de 1096 ha habido?
* Muestra los datos donde la posicion haya sido C y el evento sea G40
* Utiliza Spark SQL para mostras los registros donde el MatchID sea mayor o igual a 20

In [56]:
df = spark.read.csv("recursos/WorldCupPlayers.csv", header=True, inferSchema=True)

In [57]:
df.show()

+-------+-------+-------------+-------------------+-------+------------+-----------------+--------+---------+
|RoundID|MatchID|Team Initials|         Coach Name|Line-up|Shirt Number|      Player Name|Position|    Event|
+-------+-------+-------------+-------------------+-------+------------+-----------------+--------+---------+
|    201|   1096|          FRA|CAUDRON Raoul (FRA)|      S|           0|      Alex THEPOT|      GK|     NULL|
|    201|   1096|          MEX|   LUQUE Juan (MEX)|      S|           0|  Oscar BONFIGLIO|      GK|     NULL|
|    201|   1096|          FRA|CAUDRON Raoul (FRA)|      S|           0| Marcel LANGILLER|    NULL|     G40'|
|    201|   1096|          MEX|   LUQUE Juan (MEX)|      S|           0|     Juan CARRENO|    NULL|     G70'|
|    201|   1096|          FRA|CAUDRON Raoul (FRA)|      S|           0|  Ernest LIBERATI|    NULL|     NULL|
|    201|   1096|          MEX|   LUQUE Juan (MEX)|      S|           0|     Rafael GARZA|       C|     NULL|
|    201| 

In [58]:
df.printSchema()

root
 |-- RoundID: integer (nullable = true)
 |-- MatchID: integer (nullable = true)
 |-- Team Initials: string (nullable = true)
 |-- Coach Name: string (nullable = true)
 |-- Line-up: string (nullable = true)
 |-- Shirt Number: integer (nullable = true)
 |-- Player Name: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Event: string (nullable = true)



In [59]:
df.count()

37784

In [60]:
df.groupBy("Position").count().orderBy("count", ascending=False).show()

+--------+-----+
|Position|count|
+--------+-----+
|    NULL|33641|
|      GK| 2441|
|       C| 1510|
|     GKC|  192|
+--------+-----+



In [61]:
df.groupBy("Team Initials").count().orderBy("count", ascending=False).show()

+-------------+-----+
|Team Initials|count|
+-------------+-----+
|          BRA| 2403|
|          ITA| 1843|
|          ARG| 1807|
|          ENG| 1378|
|          FRG| 1364|
|          FRA| 1344|
|          ESP| 1317|
|          NED| 1210|
|          MEX| 1190|
|          URU| 1159|
|          GER| 1088|
|          SWE|  992|
|          BEL|  944|
|          YUG|  799|
|          SUI|  753|
|          CHI|  748|
|          USA|  729|
|          HUN|  704|
|          KOR|  695|
|          POL|  688|
+-------------+-----+
only showing top 20 rows



In [62]:
df.filter(df.MatchID == 1096).count()

33

In [63]:
df.filter((df.Position == "C") & (df.Event == "G40'")).show()

+-------+-------+-------------+--------------------+-------+------------+----------------+--------+-----+
|RoundID|MatchID|Team Initials|          Coach Name|Line-up|Shirt Number|     Player Name|Position|Event|
+-------+-------+-------------+--------------------+-------+------------+----------------+--------+-----+
|    201|   1089|          PAR|DURAND LAGUNA Jos...|      S|           0|Luis VARGAS PENA|       C| G40'|
|    429|   1175|          HUN|  DIETZ Karoly (HUN)|      S|           0|   Gyorgy SAROSI|       C| G40'|
+-------+-------+-------------+--------------------+-------+------------+----------------+--------+-----+



In [64]:
# Registrar el DataFrame como una vista temporal
df.createOrReplaceTempView("WorldCupPlayers")

# Ejecutar la consulta SQL
result = spark.sql("SELECT * FROM WorldCupPlayers WHERE MatchID >= 20")

# Mostrar los resultados
result.show()

+-------+-------+-------------+-------------------+-------+------------+-----------------+--------+---------+
|RoundID|MatchID|Team Initials|         Coach Name|Line-up|Shirt Number|      Player Name|Position|    Event|
+-------+-------+-------------+-------------------+-------+------------+-----------------+--------+---------+
|    201|   1096|          FRA|CAUDRON Raoul (FRA)|      S|           0|      Alex THEPOT|      GK|     NULL|
|    201|   1096|          MEX|   LUQUE Juan (MEX)|      S|           0|  Oscar BONFIGLIO|      GK|     NULL|
|    201|   1096|          FRA|CAUDRON Raoul (FRA)|      S|           0| Marcel LANGILLER|    NULL|     G40'|
|    201|   1096|          MEX|   LUQUE Juan (MEX)|      S|           0|     Juan CARRENO|    NULL|     G70'|
|    201|   1096|          FRA|CAUDRON Raoul (FRA)|      S|           0|  Ernest LIBERATI|    NULL|     NULL|
|    201|   1096|          MEX|   LUQUE Juan (MEX)|      S|           0|     Rafael GARZA|       C|     NULL|
|    201| 

## EJERCICIO 2

A partir del archivo nombres.json, crea un DataFrame y realiza las siguientes operaciones:

1. Crea una nueva columna (columna Mayor30) que indique si la persona es mayor de 30 años.
2. Crea una nueva columna (columna FaltanJubilacion) que calcule cuantos años le faltan para jubilarse (supongamos que se jubila a los 67 años)
3. Crea una nueva columna (columna Apellidos) que contenga XYZ (puedes utilizar la función lit)
4. Elimina las columna Mayor30 y Apellidos.
5. Crea una nueva columna (columna AnyoNac) con el año de nacimiento de cada persona (puedes utilizar la función current_date).
6. Añade un id incremental para cada fila (campo Id) y haz que al hacer un show se vea en primer lugar (puedes utilizar la función monotonically_increasing_id) seguidos del Nombre, Edad, AnyoNac, FaltaJubilacion y Ciudad

Al realizar los seis pasos, el resultado del DataFrame será similar a :
``````
+---+-------+----+-------+----------------+--------+
| Id|Nombre |Edad|AnyoNac|FaltanJubilacion|  Ciudad|
+---+-------+----+-------+----------------+--------+
|  0|  Aitor|  45|   1977|              22|   Elche|
|  1| Marina|  14|   2008|              53|Alicante|
|  2|  Laura|  19|   2003|              48|   Elche|
|  3|  Sonia|  45|   1977|              22|    Aspe|
|  4|  Pedro|null|   null|            null|   Elche|
+---+-------+----+-------+----------------+--------+
``````

In [65]:
df_nombres = spark.read.json("recursos/nombres.json")
df_nombres.show()

+--------+----+------+
|  Ciudad|Edad|Nombre|
+--------+----+------+
|   Elche|  45| Aitor|
|Alicante|  14|Marina|
|   Elche|  19| Laura|
|    Aspe|  45| Sonia|
|   Elche|NULL| Pedro|
+--------+----+------+



In [66]:
from pyspark.sql.functions import expr
# Agregar la columna Mayor30
df_nombres = df_nombres.withColumn("Mayor30", expr("Edad > 30"))
df_nombres.show()


+--------+----+------+-------+
|  Ciudad|Edad|Nombre|Mayor30|
+--------+----+------+-------+
|   Elche|  45| Aitor|   true|
|Alicante|  14|Marina|  false|
|   Elche|  19| Laura|  false|
|    Aspe|  45| Sonia|   true|
|   Elche|NULL| Pedro|   NULL|
+--------+----+------+-------+



In [67]:
# Agregar la columna FaltanJubilacion
df_nombres = df_nombres.withColumn("FaltanJubilacion", expr("67 - Edad"))

# Mostrar los resultados
df_nombres.show()


+--------+----+------+-------+----------------+
|  Ciudad|Edad|Nombre|Mayor30|FaltanJubilacion|
+--------+----+------+-------+----------------+
|   Elche|  45| Aitor|   true|              22|
|Alicante|  14|Marina|  false|              53|
|   Elche|  19| Laura|  false|              48|
|    Aspe|  45| Sonia|   true|              22|
|   Elche|NULL| Pedro|   NULL|            NULL|
+--------+----+------+-------+----------------+



In [68]:
from pyspark.sql.functions import lit

df_nombres = df_nombres.withColumn("Apellidos", lit("XYZ"))
df_nombres.show()

+--------+----+------+-------+----------------+---------+
|  Ciudad|Edad|Nombre|Mayor30|FaltanJubilacion|Apellidos|
+--------+----+------+-------+----------------+---------+
|   Elche|  45| Aitor|   true|              22|      XYZ|
|Alicante|  14|Marina|  false|              53|      XYZ|
|   Elche|  19| Laura|  false|              48|      XYZ|
|    Aspe|  45| Sonia|   true|              22|      XYZ|
|   Elche|NULL| Pedro|   NULL|            NULL|      XYZ|
+--------+----+------+-------+----------------+---------+



In [69]:
df_nombres = df_nombres.drop("Mayor30", "Apellidos")
df_nombres.show()

+--------+----+------+----------------+
|  Ciudad|Edad|Nombre|FaltanJubilacion|
+--------+----+------+----------------+
|   Elche|  45| Aitor|              22|
|Alicante|  14|Marina|              53|
|   Elche|  19| Laura|              48|
|    Aspe|  45| Sonia|              22|
|   Elche|NULL| Pedro|            NULL|
+--------+----+------+----------------+



In [70]:
from pyspark.sql.functions import current_date, year

df_nombres = df_nombres.withColumn("AnyoNac", year(current_date()) - df_nombres["Edad"])
df_nombres.show() 

+--------+----+------+----------------+-------+
|  Ciudad|Edad|Nombre|FaltanJubilacion|AnyoNac|
+--------+----+------+----------------+-------+
|   Elche|  45| Aitor|              22|   1980|
|Alicante|  14|Marina|              53|   2011|
|   Elche|  19| Laura|              48|   2006|
|    Aspe|  45| Sonia|              22|   1980|
|   Elche|NULL| Pedro|            NULL|   NULL|
+--------+----+------+----------------+-------+



In [47]:
from pyspark.sql.functions import monotonically_increasing_id

# Añadir columna de ID incremental
df_nombres = df_nombres.withColumn("Id", monotonically_increasing_id())

# Reordenar las columnas
df_nombres = df_nombres.select("Id", "Nombre", "Edad", "AnyoNac", "FaltanJubilacion", "Ciudad")

# Mostrar el resultado
df_nombres.show()


+---+------+----+-------+----------------+--------+
| Id|Nombre|Edad|AnyoNac|FaltanJubilacion|  Ciudad|
+---+------+----+-------+----------------+--------+
|  0| Aitor|  45|   1980|              22|   Elche|
|  1|Marina|  14|   2011|              53|Alicante|
|  2| Laura|  19|   2006|              48|   Elche|
|  3| Sonia|  45|   1980|              22|    Aspe|
|  4| Pedro|NULL|   NULL|            NULL|   Elche|
+---+------+----+-------+----------------+--------+



## EJERCICIO 3

A partir del archivo VentasNulos.csv:

1. Elimina las filas que tengan al menos 4 nulos.

2. Con las filas restantes, sustituye:

    * Los nombres nulos por Empleado
    * Las ventas nulas por la media de las ventas de los compañeros (redondeado a entero).
    ``````
        media = df.groupBy().avg('Ventas')
    ``````
    * Los euros nulos por el valor del compañero que menos € ha ganado. (tras agrupar, puedes usar la función min)
    * La ciudad nula por C.V. y el identificador nulo por XYZ

In [74]:
df_venta = spark.read.csv("recursos/VentasNulos.csv", header=True, inferSchema=True)
df_venta.show()

+------+------+-----+-----------+-------------+
|Nombre|Ventas|Euros|     Ciudad|Identificador|
+------+------+-----+-----------+-------------+
|  Pepe|     4|  200|      Elche|          X21|
|Andreu|     8| NULL|       NULL|         NULL|
|  Juan|  NULL| NULL|       NULL|          C54|
| Pedro|     1|   30|   Valencia|          R23|
| María|  NULL|  300| Torrellano|         NULL|
|Marina|     3|  350|       Aspe|          V55|
|  NULL|    10|  500|Crevillente|          AMV|
|   Ana|    10| 2300|   Alicante|          B89|
|  NULL|  NULL| NULL|       NULL|         NULL|
| Jorge|     8| NULL|       NULL|          T19|
+------+------+-----+-----------+-------------+



In [75]:
from pyspark.sql.functions import expr

# Crear una expresión para contar los valores nulos en cada fila
expr_nulos = """
    IF(Nombre IS NULL, 1, 0) + 
    IF(Ventas IS NULL, 1, 0) + 
    IF(Euros IS NULL, 1, 0) + 
    IF(Ciudad IS NULL, 1, 0) + 
    IF(Identificador IS NULL, 1, 0)
"""

# Filtrar las filas que tengan menos de 4 nulos
df_venta_filtrado = df_venta.filter(expr(f"({expr_nulos}) < 4"))

df_venta_filtrado.show()


+------+------+-----+-----------+-------------+
|Nombre|Ventas|Euros|     Ciudad|Identificador|
+------+------+-----+-----------+-------------+
|  Pepe|     4|  200|      Elche|          X21|
|Andreu|     8| NULL|       NULL|         NULL|
|  Juan|  NULL| NULL|       NULL|          C54|
| Pedro|     1|   30|   Valencia|          R23|
| María|  NULL|  300| Torrellano|         NULL|
|Marina|     3|  350|       Aspe|          V55|
|  NULL|    10|  500|Crevillente|          AMV|
|   Ana|    10| 2300|   Alicante|          B89|
| Jorge|     8| NULL|       NULL|          T19|
+------+------+-----+-----------+-------------+



In [78]:
from pyspark.sql.functions import expr, avg, min, round

# 1. Calcular la media de las ventas (redondeado a entero)
media_ventas = df_venta_filtrado.agg(round(avg("Ventas"), 0).alias("media_ventas")).collect()[0]["media_ventas"]

# 2. Calcular el valor mínimo de euros
min_euros = df_venta_filtrado.agg(min("Euros")).collect()[0][0]

# 3. Sustituir los valores nulos en la columna Nombre por "Empleado"
df_venta_filtrado_nombre = df_venta_filtrado.withColumn("Nombre", 
    expr("CASE WHEN Nombre IS NULL THEN 'Empleado' ELSE Nombre END"))

# 4. Sustituir los valores nulos en la columna Ciudad por "C.V."
df_venta_filtrado_ciudad = df_venta_filtrado_nombre.withColumn("Ciudad", 
    expr("CASE WHEN Ciudad IS NULL THEN 'C.V.' ELSE Ciudad END"))

# 5. Sustituir los valores nulos en la columna Identificador por "XYZ"
df_venta_filtrado_identificador = df_venta_filtrado_ciudad.withColumn("Identificador", 
    expr("CASE WHEN Identificador IS NULL THEN 'XYZ' ELSE Identificador END"))

# 6. Sustituir los valores nulos en la columna Ventas por la media de las ventas
df_venta_filtrado_ventas = df_venta_filtrado_identificador.withColumn("Ventas", 
    expr(f"CASE WHEN Ventas IS NULL THEN {media_ventas} ELSE Ventas END"))

# 7. Sustituir los valores nulos en la columna Euros por el valor mínimo de euros
df_venta_filtrado_euros = df_venta_filtrado_ventas.withColumn("Euros", 
    expr(f"CASE WHEN Euros IS NULL THEN {min_euros} ELSE Euros END"))

# Mostrar el resultado final
df_venta_filtrado_euros.show()


+--------+------+-----+-----------+-------------+
|  Nombre|Ventas|Euros|     Ciudad|Identificador|
+--------+------+-----+-----------+-------------+
|    Pepe|   4.0|  200|      Elche|          X21|
|  Andreu|   8.0|   30|       C.V.|          XYZ|
|    Juan|   6.0|   30|       C.V.|          C54|
|   Pedro|   1.0|   30|   Valencia|          R23|
|   María|   6.0|  300| Torrellano|          XYZ|
|  Marina|   3.0|  350|       Aspe|          V55|
|Empleado|  10.0|  500|Crevillente|          AMV|
|     Ana|  10.0| 2300|   Alicante|          B89|
|   Jorge|   8.0|   30|       C.V.|          T19|
+--------+------+-----+-----------+-------------+



## EJERCICIO 4

 A partir del archivo movies.tsv, crea una esquema de forma declarativa con los campos:

* interprete de tipo string
* pelicula de tipo string
* anyo de tipo int

Cada fila del fichero implica que el actor/actriz ha trabajado en dicha película en el año indicado.
1. Una vez creado el esquema, carga los datos en un DataFrame.

A continuación, mediante el DataFrame API:

2. Muestra las películas en las que ha trabajado Murphy, Eddie (I).
3. Muestra los intérpretes que aparecen tanto en Superman como en Superman II.

In [86]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Definir el esquema de forma declarativa
esquema = StructType([
    StructField("interprete", StringType(), True),
    StructField("pelicula", StringType(), True),
    StructField("anyo", IntegerType(), True)
])

In [ ]:
# Cargar el archivo movies.tsv con el esquema
df_movies = spark.read.option("delimiter", "\t").schema(esquema).csv("recursos/movies.tsv")

# Mostrar el DataFrame resultante
df_movies.show(truncate=False)

+-----------------+---------------------------+----+
|interprete       |pelicula                   |anyo|
+-----------------+---------------------------+----+
|McClure, Marc (I)|Freaky Friday              |2003|
|McClure, Marc (I)|Coach Carter               |2005|
|McClure, Marc (I)|Superman II                |1980|
|McClure, Marc (I)|Apollo 13                  |1995|
|McClure, Marc (I)|Superman                   |1978|
|McClure, Marc (I)|Back to the Future         |1985|
|McClure, Marc (I)|Back to the Future Part III|1990|
|Cooper, Chris (I)|Me, Myself & Irene         |2000|
|Cooper, Chris (I)|October Sky                |1999|
|Cooper, Chris (I)|Capote                     |2005|
|Cooper, Chris (I)|The Bourne Supremacy       |2004|
|Cooper, Chris (I)|The Patriot                |2000|
|Cooper, Chris (I)|The Town                   |2010|
|Cooper, Chris (I)|Seabiscuit                 |2003|
|Cooper, Chris (I)|A Time to Kill             |1996|
|Cooper, Chris (I)|Where the Wild Things Are  

In [93]:
# Cargar los datos en un DataFrame con el esquema definido
df_movies = spark.read.option("delimiter", "\t").schema(esquema).csv("recursos/movies.tsv")

# Mostrar el DataFrame cargado
df_movies.show(truncate=False)

+-----------------+---------------------------+----+
|interprete       |pelicula                   |anyo|
+-----------------+---------------------------+----+
|McClure, Marc (I)|Freaky Friday              |2003|
|McClure, Marc (I)|Coach Carter               |2005|
|McClure, Marc (I)|Superman II                |1980|
|McClure, Marc (I)|Apollo 13                  |1995|
|McClure, Marc (I)|Superman                   |1978|
|McClure, Marc (I)|Back to the Future         |1985|
|McClure, Marc (I)|Back to the Future Part III|1990|
|Cooper, Chris (I)|Me, Myself & Irene         |2000|
|Cooper, Chris (I)|October Sky                |1999|
|Cooper, Chris (I)|Capote                     |2005|
|Cooper, Chris (I)|The Bourne Supremacy       |2004|
|Cooper, Chris (I)|The Patriot                |2000|
|Cooper, Chris (I)|The Town                   |2010|
|Cooper, Chris (I)|Seabiscuit                 |2003|
|Cooper, Chris (I)|A Time to Kill             |1996|
|Cooper, Chris (I)|Where the Wild Things Are  

In [94]:
# Filtrar las filas donde el intérprete sea "Murphy, Eddie (I)"
df_murphy = df_movies.filter(df_movies["interprete"] == "Murphy, Eddie (I)")

# Mostrar las películas en las que ha trabajado
df_murphy.select("pelicula").show(truncate=False)


+------------------------------+
|pelicula                      |
+------------------------------+
|Showtime                      |
|Norbit                        |
|Hot Tub Time Machine          |
|Nutty Professor II: The Klumps|
|Beverly Hills Cop II          |
|Trading Places                |
|Daddy Day Care                |
|Dr. Dolittle 2                |
|Shrek Forever After           |
|Beverly Hills Cop             |
|Shrek                         |
|The Haunted Mansion           |
|Coming to America             |
|Shrek 2                       |
|Doctor Dolittle               |
|The Nutty Professor           |
|Mulan                         |
|Tower Heist                   |
|Dreamgirls                    |
|Bowfinger                     |
+------------------------------+
only showing top 20 rows



In [95]:
# Filtrar las filas donde la película sea "Superman"
df_superman = df_movies.filter(df_movies["pelicula"] == "Superman")

# Filtrar las filas donde la película sea "Superman II"
df_superman_ii = df_movies.filter(df_movies["pelicula"] == "Superman II")

# Seleccionar los intérpretes de ambas películas
intérpretes_superman = df_superman.select("interprete")
intérpretes_superman_ii = df_superman_ii.select("interprete")

# Hacer la intersección entre los intérpretes de ambas películas
interseccion = intérpretes_superman.intersect(intérpretes_superman_ii)

# Mostrar los intérpretes comunes
interseccion.show(truncate=False)


+------------------+
|interprete        |
+------------------+
|O'Halloran, Jack  |
|Tucker, Burnell   |
|Hollis, John (I)  |
|Beatty, Ned       |
|Stamp, Terence    |
|Ratzenberger, John|
|Hackman, Gene     |
|Fielder, Harry    |
|Perrine, Valerie  |
|McClure, Marc (I) |
|Donner, Richard   |
+------------------+



## EJERCICIO 5
Realiza las siguientes operaciones:
* Carga el dataset de “data/stocks_price_final.csv”, con el esquema correcto de datos (tienes que crear tu el schema").
* Renombra la variable market.cap a market
* Elimina la variable market
* Muestra las filas donde el valor de "open" es nulo.
* Elimina las filas donde el valor de "open" es nulo.
* Para comprobar el punto anterior vuelve a mostrar las filas donde el valor de "open" es nulo.


In [103]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, LongType

# Definir el esquema de forma declarativa
esquema_price = StructType([
    StructField("id", StringType(), True),
    StructField("symbol", StringType(), True),
    StructField("date", DateType(), True),
    StructField("open", FloatType(), True),
    StructField("high", FloatType(), True),
    StructField("low", FloatType(), True),
    StructField("close", FloatType(), True),
    StructField("volume", LongType(), True),
    StructField("adjusted", FloatType(), True),
    StructField("market.cap", StringType(), True),
    StructField("sector", StringType(), True),
    StructField("industry", StringType(), True),
    StructField("exchange", StringType(), True)
])

In [105]:
df_price = spark.read.csv("recursos/stocks_price_final.csv", schema=esquema_price)
df_price.show()

+----+------+----------+-----+------+------+-----+-------+--------+----------+-------------+--------------------+--------+
|  id|symbol|      date| open|  high|   low|close| volume|adjusted|market.cap|       sector|            industry|exchange|
+----+------+----------+-----+------+------+-----+-------+--------+----------+-------------+--------------------+--------+
|NULL|symbol|      NULL| NULL|  NULL|  NULL| NULL|   NULL|    NULL|market.cap|       sector|            industry|exchange|
|   1|   TXG|2019-09-12| 54.0|  58.0|  51.0|52.75|7326300|   52.75|    $9.31B|Capital Goods|Biotechnology: La...|  NASDAQ|
|   2|   TXG|2019-09-13|52.75|54.355| 49.15|52.27|1025200|   52.27|    $9.31B|Capital Goods|Biotechnology: La...|  NASDAQ|
|   3|   TXG|2019-09-16|52.45|  56.0| 52.01| 55.2| 269900|    55.2|    $9.31B|Capital Goods|Biotechnology: La...|  NASDAQ|
|   4|   TXG|2019-09-17|56.21|  60.9|55.423|56.78| 602800|   56.78|    $9.31B|Capital Goods|Biotechnology: La...|  NASDAQ|
|   5|   TXG|201

In [107]:
# Renombrar la columna market.cap a market
df_renombrado = df_price.withColumnRenamed("market.cap", "market")

# Mostrar el DataFrame con el nuevo nombre de la columna
df_renombrado.show()

+----+------+----------+-----+------+------+-----+-------+--------+----------+-------------+--------------------+--------+
|  id|symbol|      date| open|  high|   low|close| volume|adjusted|    market|       sector|            industry|exchange|
+----+------+----------+-----+------+------+-----+-------+--------+----------+-------------+--------------------+--------+
|NULL|symbol|      NULL| NULL|  NULL|  NULL| NULL|   NULL|    NULL|market.cap|       sector|            industry|exchange|
|   1|   TXG|2019-09-12| 54.0|  58.0|  51.0|52.75|7326300|   52.75|    $9.31B|Capital Goods|Biotechnology: La...|  NASDAQ|
|   2|   TXG|2019-09-13|52.75|54.355| 49.15|52.27|1025200|   52.27|    $9.31B|Capital Goods|Biotechnology: La...|  NASDAQ|
|   3|   TXG|2019-09-16|52.45|  56.0| 52.01| 55.2| 269900|    55.2|    $9.31B|Capital Goods|Biotechnology: La...|  NASDAQ|
|   4|   TXG|2019-09-17|56.21|  60.9|55.423|56.78| 602800|   56.78|    $9.31B|Capital Goods|Biotechnology: La...|  NASDAQ|
|   5|   TXG|201

In [108]:
# Eliminar la columna market
df_sin_market = df_renombrado.drop("market")

# Mostrar el DataFrame sin la columna market
df_sin_market.show()

+----+------+----------+-----+------+------+-----+-------+--------+-------------+--------------------+--------+
|  id|symbol|      date| open|  high|   low|close| volume|adjusted|       sector|            industry|exchange|
+----+------+----------+-----+------+------+-----+-------+--------+-------------+--------------------+--------+
|NULL|symbol|      NULL| NULL|  NULL|  NULL| NULL|   NULL|    NULL|       sector|            industry|exchange|
|   1|   TXG|2019-09-12| 54.0|  58.0|  51.0|52.75|7326300|   52.75|Capital Goods|Biotechnology: La...|  NASDAQ|
|   2|   TXG|2019-09-13|52.75|54.355| 49.15|52.27|1025200|   52.27|Capital Goods|Biotechnology: La...|  NASDAQ|
|   3|   TXG|2019-09-16|52.45|  56.0| 52.01| 55.2| 269900|    55.2|Capital Goods|Biotechnology: La...|  NASDAQ|
|   4|   TXG|2019-09-17|56.21|  60.9|55.423|56.78| 602800|   56.78|Capital Goods|Biotechnology: La...|  NASDAQ|
|   5|   TXG|2019-09-18|56.85| 62.27| 55.65| 62.0|1589600|    62.0|Capital Goods|Biotechnology: La...|  

In [109]:
# Filtrar filas donde el valor de "open" es nulo
df_sin_market.filter(df_sin_market["open"].isNull()).show()

+----+------+----------+----+----+----+-----+------+--------+-------------+--------------------+--------+
|  id|symbol|      date|open|high| low|close|volume|adjusted|       sector|            industry|exchange|
+----+------+----------+----+----+----+-----+------+--------+-------------+--------------------+--------+
|NULL|symbol|      NULL|NULL|NULL|NULL| NULL|  NULL|    NULL|       sector|            industry|exchange|
|4378|  KRKR|2020-05-11|NULL|NULL|NULL| NULL|  NULL|    NULL|Miscellaneous|   Business Services|  NASDAQ|
|5747|  NMTR|2020-01-23|NULL|NULL|NULL| NULL|  NULL|    NULL|  Health Care|Major Pharmaceuti...|  NASDAQ|
|5748|  NMTR|2020-01-24|NULL|NULL|NULL| NULL|  NULL|    NULL|  Health Care|Major Pharmaceuti...|  NASDAQ|
|5749|  NMTR|2020-01-27|NULL|NULL|NULL| NULL|  NULL|    NULL|  Health Care|Major Pharmaceuti...|  NASDAQ|
|5750|  NMTR|2020-01-28|NULL|NULL|NULL| NULL|  NULL|    NULL|  Health Care|Major Pharmaceuti...|  NASDAQ|
|5751|  NMTR|2020-01-29|NULL|NULL|NULL| NULL| 

In [110]:
# Eliminar filas donde el valor de "open" es nulo
df_sin_open_nulo = df_sin_market.filter(df_sin_market["open"].isNotNull())

# Mostrar el resultado
df_sin_open_nulo.show()

+---+------+----------+-----+------+------+-----+-------+--------+-------------+--------------------+--------+
| id|symbol|      date| open|  high|   low|close| volume|adjusted|       sector|            industry|exchange|
+---+------+----------+-----+------+------+-----+-------+--------+-------------+--------------------+--------+
|  1|   TXG|2019-09-12| 54.0|  58.0|  51.0|52.75|7326300|   52.75|Capital Goods|Biotechnology: La...|  NASDAQ|
|  2|   TXG|2019-09-13|52.75|54.355| 49.15|52.27|1025200|   52.27|Capital Goods|Biotechnology: La...|  NASDAQ|
|  3|   TXG|2019-09-16|52.45|  56.0| 52.01| 55.2| 269900|    55.2|Capital Goods|Biotechnology: La...|  NASDAQ|
|  4|   TXG|2019-09-17|56.21|  60.9|55.423|56.78| 602800|   56.78|Capital Goods|Biotechnology: La...|  NASDAQ|
|  5|   TXG|2019-09-18|56.85| 62.27| 55.65| 62.0|1589600|    62.0|Capital Goods|Biotechnology: La...|  NASDAQ|
|  6|   TXG|2019-09-19|62.81|63.375| 61.03|61.12| 425200|   61.12|Capital Goods|Biotechnology: La...|  NASDAQ|
|

In [112]:
# Mostrar cuantas filas tienen el valor de "open" es nulo 
df_sin_open_nulo.filter(df_sin_open_nulo["open"].isNull()).count()

0